<a href="https://colab.research.google.com/github/nittayac/DFEDATA6-EX1/blob/main/04_Image%20Analysis_HelloAPIs.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [72]:
!pip install --upgrade azure-cognitiveservices-vision-computervision
!pip install pillow

In [73]:
!pip install azure-storage-blob

     |████████████████████████████████| 356 kB 4.3 MB/s 
     |████████████████████████████████| 178 kB 63.8 MB/s 
     |████████████████████████████████| 3.6 MB 59.5 MB/s 


In [4]:
from azure.cognitiveservices.vision.computervision import ComputerVisionClient
from azure.cognitiveservices.vision.computervision.models import OperationStatusCodes
from azure.cognitiveservices.vision.computervision.models import VisualFeatureTypes
from azure.storage.blob import BlobServiceClient, BlobClient, ContainerClient, __version__
from msrest.authentication import CognitiveServicesCredentials

from array import array
import os , uuid
from PIL import Image
import sys
import time

In [24]:
#subscription_key = "7db033dbbbc141e1af52f567e4c9cc36"
#endpoint = "https://popopopopo.cognitiveservices.azure.com/"


subscription_key = "59b0d8d0ba5647de95962d9f460fc9e6"
endpoint = "https://imageanalysis2022.cognitiveservices.azure.com/"

In [25]:
creds= CognitiveServicesCredentials(subscription_key)
computervision_client = ComputerVisionClient(endpoint, creds)
# API Authentication settings

In [27]:
#connect blob storage
conn_str = 'DefaultEndpointsProtocol=https;AccountName=images20222;AccountKey=po/pxwEuUrCUOYy3ubyeW+IEIm4TTUh2SX4dZiGiWVIT9yTBZk74Vcw6v59Pi79QTLzs44ZxYPPe+AStczfOHA==;EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(conn_str)
conn_str = 'DefaultEndpointsProtocol=https;AccountName=images20222;AccountKey=po/pxwEuUrCUOYy3ubyeW+IEIm4TTUh2SX4dZiGiWVIT9yTBZk74Vcw6v59Pi79QTLzs44ZxYPPe+AStczfOHA==;EndpointSuffix=core.windows.net'
blob_service_client = BlobServiceClient.from_connection_string(conn_str)

In [36]:
#read data from blob storage : Solution 1

try:
    print("Azure Blob Storage v" + __version__ + " - Python quickstart sample")

    # Quick start code goes here

except Exception as ex:
    print('Exception:')
    print(ex)

contraner_name = "images"
blob_list = blob_service_client.get_container_client(contraner_name).list_blobs()
url = blob_service_client.url

blobImages = []
for blob in blob_list:
    blobImages.append(url+contraner_name+"/"+blob.name)

print(blobImages)


Azure Blob Storage v12.9.0 - Python quickstart sample
['https://images20222.blob.core.windows.net/images/0001.jpg', 'https://images20222.blob.core.windows.net/images/0002.jpg', 'https://images20222.blob.core.windows.net/images/0003.jpg']


In [30]:
#read data from blob storage : Solution 2

from posix import listdir
#baseimgurl = 'https://dodokilledplatypus.blob.core.windows.net/images/0000'

baseimgurl = 'https://images20222.blob.core.windows.net/images/000'  # contrainer : images , should be public 

#Enable contrainer as public (Home >> imagess20222(Stirage Name) >> Contraner >> check box : image (Contraner) >> Change access level >> Contrainer(anonymous read access for contrainer and blobs))

allurls = []
for i in range(3):
  filename = baseimgurl + str(i+1) + '.jpg'
  allurls.append(filename)

allurls

['https://images20222.blob.core.windows.net/images/0001.jpg',
 'https://images20222.blob.core.windows.net/images/0002.jpg',
 'https://images20222.blob.core.windows.net/images/0003.jpg']

In [58]:
'''
Describe an Image - remote
This example describes the contents of an image with the confidence score.
'''
print("===== Describe an image - remote =====")
# Call API

for url in allurls:
  description_results =  computervision_client.describe_image(url)

  print("Description of remote image: ")
  if (len(description_results.captions) == 0):
      print("No description detected.")
  else:
      for caption in description_results.captions:
          print("'{}' with confidence {:.2f}%".format(caption.text, caption.confidence * 100))
  print('**************************')
# Get the captions (descriptions) from the response, with confidence level


===== Describe an image - remote =====
Description of remote image: 
'a person in a suit and bow tie with Matt Smith et al. behind him' with confidence 50.23%
**************************
Description of remote image: 
'a group of toys' with confidence 38.06%
**************************
Description of remote image: 
'a house with a garage' with confidence 53.95%
**************************


In [59]:
'''
Categorize an Image - remote
This example extracts (general) categories from a remote image with a confidence score.
'''
print("===== Categorize an image - remote =====")
# Select the visual feature(s) you want.
remote_image_features = ["categories"]
# Call API with URL and features

for url in allurls:
  categorize_results_remote = computervision_client.analyze_image(url , remote_image_features)
  # Print results with confidence score
  print("Categories from remote image: ")
  if (len(categorize_results_remote.categories) == 0):
      print("No categories detected.")
  else:
      for category in categorize_results_remote.categories:
          print("'{}' with confidence {:.2f}%".format(category.name, category.score * 100))
  print('**********')

===== Categorize an image - remote =====
Categories from remote image: 
'people_group' with confidence 48.44%
**********
Categories from remote image: 
No categories detected.
**********
Categories from remote image: 
'building_' with confidence 66.80%
'outdoor_house' with confidence 24.22%
**********


In [60]:
'''
Tag an Image - remote
This example returns a tag (key word) for each thing in the image.
'''
print("===== Tag an image - remote =====")
# Call API with remote image
for url in allurls:
  tags_result_remote = computervision_client.tag_image(url )

  # Print results with confidence score
  print("Tags in the remote image: ")
  if (len(tags_result_remote.tags) == 0):
      print("No tags detected.")
  else:
      for tag in tags_result_remote.tags:
          print("'{}' with confidence {:.2f}%".format(tag.name, tag.confidence * 100))

  print('*************')

===== Tag an image - remote =====
Tags in the remote image: 
'human face' with confidence 99.61%
'person' with confidence 99.55%
'clothing' with confidence 99.28%
'man' with confidence 92.46%
'painting' with confidence 87.29%
*************
Tags in the remote image: 
'cartoon' with confidence 98.23%
'animated cartoon' with confidence 93.99%
'animation' with confidence 92.06%
'tree' with confidence 90.55%
*************
Tags in the remote image: 
'building' with confidence 99.83%
'outdoor' with confidence 99.41%
'house' with confidence 99.21%
'property' with confidence 99.16%
'home' with confidence 98.59%
'real estate' with confidence 98.27%
'window' with confidence 97.07%
'siding' with confidence 96.24%
'sky' with confidence 96.01%
'plant' with confidence 95.71%
'grass' with confidence 94.48%
'cottage' with confidence 92.74%
'door' with confidence 92.59%
'yard' with confidence 92.28%
'roof' with confidence 91.84%
'farmhouse' with confidence 91.11%
'porch' with confidence 90.31%
'driveway

In [61]:
'''
Detect Faces - remote
This example detects faces in a remote image, gets their gender and age, 
and marks them with a bounding box.
'''


print("===== Detect Faces - remote =====")
# Get an image with faces
# Select the visual feature(s) you want.
remote_image_features = ["faces"]
# Call the API with remote URL and features

for url in allurls:
  detect_faces_results_remote = computervision_client.analyze_image(url, remote_image_features)

  # Print the results with gender, age, and bounding box
  print("Faces in the remote image: ")
  if (len(detect_faces_results_remote.faces) == 0):
      print("No faces detected.")
  else:
      for face in detect_faces_results_remote.faces:
          print("'{}' of age {} at location {}, {}, {}, {}".format(face.gender, face.age, \
          face.face_rectangle.left, face.face_rectangle.top, \
          face.face_rectangle.left + face.face_rectangle.width, \
          face.face_rectangle.top + face.face_rectangle.height))
  print('***********')

===== Detect Faces - remote =====
Faces in the remote image: 
'Male' of age 30 at location 182, 50, 251, 119
'Male' of age 32 at location 270, 63, 322, 115
***********
Faces in the remote image: 
No faces detected.
***********
Faces in the remote image: 
No faces detected.
***********
